In [1]:
import torch
import torchvision
import numpy as np
from PIL import Image, ImageDraw
!pip install onnx onnxruntime
import onnx
import onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 614.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.9 MB/s eta 0:00:00


In [2]:
# Установка пакета из форка репозитория onnx2torch
#!pip uninstall onnx2torch
#!pip install onnx2torch
!pip install -e git+https://github.com/KaizerloveRussia/onnx2torch.git#egg=onnx2torch

Obtaining onnx2torch from git+https://github.com/KaizerloveRussia/onnx2torch.git#egg=onnx2torch
  Cloning https://github.com/KaizerloveRussia/onnx2torch.git to ./src/onnx2torch
  Running command git clone --filter=blob:none --quiet https://github.com/KaizerloveRussia/onnx2torch.git /content/src/onnx2torch
  Resolved https://github.com/KaizerloveRussia/onnx2torch.git to commit ad46e9224f77ccaa06e5a327bc7269aac0778aba
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_c

In [3]:
# Загрузка и предобработка изображения
from google.colab import files
uploaded=files.upload()
image = Image.open("image.png").convert('RGB')
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
image1 = image
image = transform(image)
image = image.unsqueeze(0)

Saving image.png to image.png


In [5]:
# Загрузка и запуск предобученной модели Faster R-CNN
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()
output=model(image)
print(output)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[{'boxes': tensor([[204.3542,  37.5309, 421.1374, 623.8053],
        [ 19.3879,  73.0953, 223.6135, 511.5916],
        [460.2913, 413.0642, 586.8542, 500.8302],
        [482.1385, 338.9479, 545.5147, 381.8059],
        [251.6486,  46.9328, 382.8713, 262.4328],
        [ 68.5421,  38.3235, 339.7109, 632.1019],
        [146.2069, 124.2338, 224.9680, 474.4522],
        [ 12.2086, 168.0689, 183.7289, 323.8441],
        [270.9537,  44.9405, 333.2106, 184.0173],
        [263.7951, 210.5962, 332.9884, 239.9615],
        [ 10.4931, 108.0453, 188.1809, 339.8682],
        [ 51.1781, 279.1052, 178.2148, 624.9517],
        [208.6113, 125.0314, 376.4835, 347.4301],
        [ 37.6162, 274.2091, 174.9908, 632.8361],
        [296.0182, 206.2048, 331.9722, 243.0341],
        [212.9393,  49.1574, 433.8777, 359.3833],
        [ 94.9690, 370.9078, 167.1342, 635.1591]], grad_fn=<StackBackward0>), 'labels': tensor([ 1,  1, 15,  3,  1,  1,  1,  9, 14, 35,  1,  1, 27, 64, 35,  1, 86]), 'scores': tensor([0.998

In [11]:
# Конвертация в ONNX
torch.onnx.export(model, image, "faster_rcnn.onnx", opset_version=13)

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4009: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  (torch.floor((input.size(i + 2).float() * torch.tensor(scale_factors[i], dtype=torch.float32)).float()))
/usr/local/lib/python3.10/dist-packages/torchvision/ops/boxes.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_x = torch.min(boxes_x, torch.tensor(width, dtype=boxes.dtype, device=boxes.device))
/usr/local/lib/python3.10/dist-packages/torchvision/ops/boxes.py:168: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bo

In [13]:
# Загрузка и запуск модели в ONNX
onnx_model = onnx.load("faster_rcnn.onnx")
ort_session = onnxruntime.InferenceSession("faster_rcnn.onnx")
ort_inputs = {ort_session.get_inputs()[0].name: image.numpy()}
ort_outs = ort_session.run(None, ort_inputs)
print(ort_outs)

[array([[204.35437 ,  37.53126 , 421.13736 , 623.80493 ],
       [ 19.387602,  73.09536 , 223.6132  , 511.59204 ],
       [460.29126 , 413.0641  , 586.8541  , 500.8302  ],
       [482.13867 , 338.9479  , 545.5147  , 381.80582 ],
       [251.64879 ,  46.932373, 382.87115 , 262.43295 ],
       [ 68.54188 ,  38.323658, 339.71072 , 632.10205 ],
       [146.20686 , 124.23339 , 224.96788 , 474.4528  ],
       [ 12.208509, 168.069   , 183.72903 , 323.84445 ],
       [270.95374 ,  44.94053 , 333.2105  , 184.01724 ],
       [263.79504 , 210.59636 , 332.98837 , 239.96153 ],
       [ 10.493082, 108.04506 , 188.18098 , 339.86856 ],
       [ 51.17838 , 279.10526 , 178.21457 , 624.9518  ],
       [208.61127 , 125.03099 , 376.48337 , 347.4305  ],
       [ 37.616264, 274.209   , 174.9906  , 632.8363  ],
       [296.01822 , 206.20488 , 331.9722  , 243.03407 ],
       [212.9397  ,  49.157433, 433.8778  , 359.3833  ],
       [ 94.96884 , 370.90833 , 167.13414 , 635.15924 ]], dtype=float32), array([ 1,  1

In [17]:
# Обратная конвертация
!pip install onnx2torch
from onnx2torch.converter import convert
torch_model = convert("faster_rcnn.onnx")

ModuleNotFoundError: No module named 'onnx2torch'